In [1]:
from inference.data_analysis import *

/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# target_props = ["dft_e_hull", "Op_band_center"]
target_prop = "Op_band_center"

# test_set_types = ["test_set", "holdout_set_B_sites", "holdout_set_series"]
test_set_types = ["holdout_set_series"]

# "dft_e_hull"
# experimental_settings = [
#     {"model_type": "CGCNN", "interpolation": False, "struct_type": "unrelaxed", "exp_id": 837611},
#     {"model_type": "CGCNN", "interpolation": False, "struct_type": "relaxed", "exp_id": 837612},
#     {"model_type": "CGCNN", "interpolation": False, "struct_type": "M3Gnet_relaxed", "exp_id": 837613},
#     {"model_type": "e3nn", "interpolation": False, "struct_type": "unrelaxed", "exp_id": 837627},
#     {"model_type": "e3nn", "interpolation": False, "struct_type": "relaxed", "exp_id": 837628},
#     {"model_type": "e3nn", "interpolation": False, "struct_type": "M3Gnet_relaxed", "exp_id": 837629},
#     {"model_type": "Painn", "interpolation": False, "struct_type": "unrelaxed", "exp_id": 837609},
# ]

# "Op_band_center"
experimental_settings = [
    {"model_type": "CGCNN", "interpolation": False, "struct_type": "unrelaxed", "exp_id": 837614},
    {"model_type": "CGCNN", "interpolation": False, "struct_type": "relaxed", "exp_id": 837615},
    {"model_type": "CGCNN", "interpolation": False, "struct_type": "M3Gnet_relaxed", "exp_id": 837616},
    {"model_type": "e3nn", "interpolation": False, "struct_type": "unrelaxed", "exp_id": 837624},
    {"model_type": "e3nn", "interpolation": False, "struct_type": "relaxed", "exp_id": 837625},
    {"model_type": "e3nn", "interpolation": False, "struct_type": "M3Gnet_relaxed", "exp_id": 837626},
    {"model_type": "Painn", "interpolation": False, "struct_type": "unrelaxed", "exp_id": 837610},
]

series = [
    [["Bi", "La"], ["Cu", "Mo"]],
    [["La", "Sr"], ["Co", "Fe"]],
    [["K", "Ba"], ["Ti", "Al"]],
    [["La", "Pr"], ["Y", "Ni"]],
    [["Y", "La"], ["In", "Mg"]],
    [["Mg", "Pr"], ["Ni", "V"]]
]

In [3]:
# plot_hex_all(target_prop, test_set_types, experimental_settings)

In [4]:
# plot_violin_all(target_prop, experimental_settings, series)

Completed Op_band_center_htvs_data_unrelaxed_CGCNN 837614
Completed Op_band_center_htvs_data_unrelaxed_CGCNN 837614
Completed Op_band_center_htvs_data_unrelaxed_CGCNN 837614
Completed Op_band_center_htvs_data_unrelaxed_CGCNN 837614
Completed Op_band_center_htvs_data_unrelaxed_CGCNN 837614
Completed Op_band_center_htvs_data_unrelaxed_CGCNN 837614
Completed Op_band_center_htvs_data_relaxed_CGCNN 837615
Completed Op_band_center_htvs_data_relaxed_CGCNN 837615
Completed Op_band_center_htvs_data_relaxed_CGCNN 837615
Completed Op_band_center_htvs_data_relaxed_CGCNN 837615
Completed Op_band_center_htvs_data_relaxed_CGCNN 837615
Completed Op_band_center_htvs_data_relaxed_CGCNN 837615
Completed Op_band_center_htvs_data_M3Gnet_relaxed_CGCNN 837616
Completed Op_band_center_htvs_data_M3Gnet_relaxed_CGCNN 837616
Completed Op_band_center_htvs_data_M3Gnet_relaxed_CGCNN 837616
Completed Op_band_center_htvs_data_M3Gnet_relaxed_CGCNN 837616
Completed Op_band_center_htvs_data_M3Gnet_relaxed_CGCNN 837616
C

In [5]:
# plot_training_e3nn_contrastive(target_prop, experimental_settings)

In [6]:
# num_best_models = 3
# test_set_type = "holdout_set_B_sites"

# for experimental_setting in experimental_settings:
#     sigopt_name = build_sigopt_name("data/", target_prop, experimental_setting["struct_type"], experimental_setting["interpolation"], experimental_setting["model_type"])
#     directory = "./best_models/" + experimental_setting["model_type"] + "/" + sigopt_name + "/" +str(experimental_setting["exp_id"])
#     test_set_dfs = []
    
#     for i in range(num_best_models):
#         with open(directory + "/" + "best_" + str(i) + "/" + test_set_type + "_predictions.json") as f:
#             test_set_dfs.append(pd.read_json(f).sort_values(by=["formula", "unrelaxed_cryst_id"]))

#     compositions = list(set(test_set_dfs[0]["formula"]))

#     # all_predicted_layered_rocksalt_diff = []
#     # all_true_layered_rocksalt_diff = []
#     all_predicted_energies_vs_groundstate = []
#     all_true_energies_vs_groundstate = []

#     for i in range(num_best_models):
#         # predicted_layered_rocksalt_diff = []
#         # true_layered_rocksalt_diff = []
#         predicted_energies_vs_groundstate = []
#         true_energies_vs_groundstate = []

#         for composition in compositions:
#             selected_df = test_set_dfs[i].loc[test_set_dfs[i]["formula"] == composition]
            
#             adjusted_predicted_energies = list(np.array(selected_df["predicted_" + target_prop]) - selected_df["predicted_" + target_prop].min())
#             predicted_energies_vs_groundstate = predicted_energies_vs_groundstate + adjusted_predicted_energies
#             adjusted_true_energies = list(np.array(selected_df[target_prop]) - selected_df[target_prop].min())
#             true_energies_vs_groundstate = true_energies_vs_groundstate + adjusted_true_energies
            
#             ids = list(selected_df["unrelaxed_cryst_id"])
#             # predicted_layered = selected_df.loc[selected_df["unrelaxed_cryst_id"] == min(ids)]["predicted_" + target_prop]
#             # predicted_rocksalt = selected_df.loc[selected_df["unrelaxed_cryst_id"] == max(ids)]["predicted_" + target_prop]
#             # true_layered = selected_df.loc[selected_df["unrelaxed_cryst_id"] == min(ids)][target_prop]
#             # true_rocksalt = selected_df.loc[selected_df["unrelaxed_cryst_id"] == max(ids)][target_prop]
#             # predicted_layered_rocksalt_diff.append(float(predicted_layered) - float(predicted_rocksalt))
#             # true_layered_rocksalt_diff.append(float(true_layered) - float(true_rocksalt))

#         # all_predicted_layered_rocksalt_diff.append(predicted_layered_rocksalt_diff)
#         # all_true_layered_rocksalt_diff.append(true_layered_rocksalt_diff)
#         all_predicted_energies_vs_groundstate.append(predicted_energies_vs_groundstate)
#         all_true_energies_vs_groundstate.append(true_energies_vs_groundstate)

#     # all_predicted_layered_rocksalt_diff = np.array(all_predicted_layered_rocksalt_diff).T
#     # all_true_layered_rocksalt_diff = np.array(all_true_layered_rocksalt_diff).T
#     all_predicted_energies_vs_groundstate = np.array(all_predicted_energies_vs_groundstate).T
#     all_true_energies_vs_groundstate = np.array(all_true_energies_vs_groundstate).T

#     # all_predicted_layered_rocksalt_diff_mean = np.mean(all_predicted_layered_rocksalt_diff, axis=1)
#     # all_true_layered_rocksalt_diff_mean = np.mean(all_true_layered_rocksalt_diff, axis=1)

#     # maes = []
#     # for i in range(num_best_models):
#     #     maes.append(mean_absolute_error(all_true_layered_rocksalt_diff[:,i], all_predicted_layered_rocksalt_diff[:,i]))
#     # mae_mean = np.mean(maes)
#     # mae_std = np.std(maes)

#     # plot_hex(
#     #     target_prop, all_true_layered_rocksalt_diff_mean, all_predicted_layered_rocksalt_diff_mean, test_set_type, 
#     #     experimental_setting, sigopt_name, str(experimental_setting["exp_id"]), 
#     #     additional_string="layered_rocksalt",mae_mean=mae_mean, mae_std=mae_std
#     #     )
    
#     all_predicted_energies_vs_groundstate_mean = np.mean(all_predicted_energies_vs_groundstate, axis=1)
#     all_true_energies_vs_groundstate_mean = np.mean(all_true_energies_vs_groundstate, axis=1)

#     maes = []
#     for i in range(num_best_models):
#         maes.append(mean_absolute_error(all_true_energies_vs_groundstate[:,i], all_predicted_energies_vs_groundstate[:,i]))
#     mae_mean = np.mean(maes)
#     mae_std = np.std(maes)

#     plot_hex(
#         target_prop, all_true_energies_vs_groundstate_mean, all_predicted_energies_vs_groundstate_mean, test_set_type,
#         experimental_setting, sigopt_name, str(experimental_setting["exp_id"]),
#         additional_string="energies_vs_groundstate", mae_mean=mae_mean, mae_std=mae_std
#         )